In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

In [27]:
path_main = '/content/drive/MyDrive/hack_digit/'
path_short = f'{path_main}train_dataset_Датасет.xlsx'
path_long = f'{path_main}хакатон.xlsx'
df_short = pd.read_excel(path_short)
df_long = pd.read_excel(path_long)


# Эмбеддинги

In [28]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, KeyedVectors
from sklearn.metrics import average_precision_score



In [29]:
# 1.1 TF-IDF Embeddings
def get_tfidf_embeddings(data):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(data)
    return tfidf_matrix

In [30]:
# 1.2 Word2Vec Embeddings
def get_word2vec_embeddings(data, model):
    def get_vector(text):
        words = [word for word in text.split() if word in model.wv.key_to_index]
        if len(words) == 0:
            return np.zeros(model.vector_size)
        return np.mean([model.wv[word] for word in words], axis=0)

    embeddings = np.array([get_vector(text) for text in data])
    return embeddings


## Similarity search

In [31]:
# 2. Similarity Search
def get_similar_docs(embeddings):
    cosine_similarities = cosine_similarity(embeddings)
    similar_docs = cosine_similarities.argsort()[:,::-1]
    return similar_docs


## Metrics

In [77]:
def average_precision_score(retrieved_flags, true_flags, k=None):
    """Compute the average precision."""
    if k is not None:
        retrieved_flags = retrieved_flags[:k]
        true_flags = true_flags[:k]

    ap = 0.0
    rel_count = 0
    for i, flag in enumerate(retrieved_flags):
        if flag == 1 and true_flags[i] == 1:
            rel_count += 1
            ap += rel_count / (i + 1)

    # Handle the case when there are no relevant docs
    if rel_count == 0:
        return 0

    return ap / rel_count

def mean_avg_precision_at_k(similar_docs, true_docs, k):
    scores = []
    for i in range(len(similar_docs)):
        relevant_docs = set(true_docs[i])
        retrieved_docs = similar_docs[i]
        scores.append(average_precision_score([1 if doc in relevant_docs else 0 for doc in retrieved_docs], [1]*len(retrieved_docs), k))
    return np.mean(scores)


In [33]:
from gensim.models import Word2Vec

# Preprocess data for Word2Vec
def preprocess_for_w2v(data):
    # Tokenize and filter non-alphabetic tokens
    return [ [word for word in question.lower().split() if word.isalpha()] for question in data]

# Train Word2Vec
def train_word2vec(data, vector_size=100, window=5, min_count=1, workers=4):
    return Word2Vec(data, vector_size=vector_size, window=window, min_count=min_count, workers=workers)



# Assuming df_short is the dataset and previous functions are defined
questions = df_short['QUESTION'].values

# Preprocess data and train Word2Vec
questions_for_w2v = preprocess_for_w2v(questions)
w2v_model = train_word2vec(questions_for_w2v)

# ... [Rest of the code remains unchanged]


In [78]:
# Assuming df_short is the dataset
questions = df_short['QUESTION'].values

# TF-IDF
tfidf_embeddings = get_tfidf_embeddings(questions)
tfidf_similar_docs = get_similar_docs(tfidf_embeddings)

# Word2Vec (assuming a pretrained model is loaded as w2v_model)
w2v_embeddings = get_word2vec_embeddings(questions, w2v_model)
w2v_similar_docs = get_similar_docs(w2v_embeddings)
k = 3
# Evaluating
true_docs = np.array([list(range(len(questions))) for _ in range(len(questions))])  # Assuming the entire dataset as relevant
tfidf_map = mean_avg_precision_at_k(tfidf_similar_docs, true_docs,3)
w2v_map = mean_avg_precision_at_k(w2v_similar_docs, true_docs,3)

print("TF-IDF MAP:", tfidf_map)
print("Word2Vec MAP:", w2v_map)

TF-IDF MAP: 1.0
Word2Vec MAP: 1.0


In [98]:
# Evaluation Metrics

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def recall_at_k(r, k, all_relevant):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.sum(r) / all_relevant

# Calculate metrics for a given similarity matrix and true docs
def evaluate(similar_docs, true_docs, k):
    ndcgs, precisions, recalls = [], [], []
    for i in range(len(similar_docs)):
        relevant_docs = set(true_docs[i])
        retrieved_docs = [1 if doc in relevant_docs else 0 for doc in similar_docs[i]]

        ndcgs.append(ndcg_at_k(retrieved_docs, k))
        precisions.append(precision_at_k(retrieved_docs, k))
        recalls.append(recall_at_k(retrieved_docs, k, len(relevant_docs)))

    return np.mean(ndcgs), np.mean(precisions), np.mean(recalls)

# Assuming df_short is the dataset and previous functions are defined
questions = df_short['QUESTION'].values

# TF-IDF
tfidf_embeddings = get_tfidf_embeddings(questions)
tfidf_similar_docs = get_similar_docs(tfidf_embeddings)

# Word2Vec (assuming a pretrained model is loaded as w2v_model)
w2v_embeddings = get_word2vec_embeddings(questions, w2v_model)
w2v_similar_docs = get_similar_docs(w2v_embeddings)

# Evaluating with k=10 for example
k = 3
true_docs = np.array([list(range(len(questions))) for _ in range(len(questions))])  # Assuming the entire dataset as relevant

tfidf_ndcg, tfidf_precision, tfidf_recall = evaluate(tfidf_similar_docs, true_docs, k)
w2v_ndcg, w2v_precision, w2v_recall = evaluate(w2v_similar_docs, true_docs, k)

print(f"TF-IDF NDCG@{k}:", tfidf_ndcg)
print(f"TF-IDF Precision@{k}:", tfidf_precision)
print(f"TF-IDF Recall@{k}:", tfidf_recall)

print(f"Word2Vec NDCG@{k}:", w2v_ndcg)
print(f"Word2Vec Precision@{k}:", w2v_precision)
print(f"Word2Vec Recall@{k}:", w2v_recall)


TF-IDF NDCG@3: 1.0
TF-IDF Precision@3: 1.0
TF-IDF Recall@3: 0.0010976948408342481
Word2Vec NDCG@3: 1.0
Word2Vec Precision@3: 1.0
Word2Vec Recall@3: 0.0010976948408342481


# Переформулированные

In [89]:
df_test = pd.read_excel('test_questions.xlsx')

In [90]:
df_test

,Оригинал вопроса,Ответ,Переформулированный 1,Переформулированный 2
0,Какие основания для получения 33 услуги?,Заявителем является родитель (усыновитель) реб...,"Каковы требования для того, чтобы воспользоват...",Под какими условиями можно получить 33-ю услуг...
1,Какие условия являются обязательными для получ...,"ребенок - гражданин РФ, имеющий место жительст...",Каковы требования для получения единовременной...,Под какими условиями могу претендовать на соци...
2,Как оформлять детскую карту при рождении (усын...,При наличии в семье двух и более детей единовр...,Каков процесс оформления детской карты в случа...,Какие шаги следует предпринять для оформления ...
3,Кому не положена детская карта?,"Гражданам, дети которых находятся на полном го...",Какие категории граждан не могут претендовать ...,Для каких граждан выдача детской карты не пред...
4,Как определяется размер по единовременной ком...,Для определения размера единовременной компенс...,Какие факторы учитываются при расчете размера ...,Какие критерии определены для расчета суммы ко...
5,Какие дополнительно документы должен предостав...,"1. Справка, подтверждающая постановку на медиц...",Какой дополнительный пакет документов требуетс...,Какие документы необходимо собрать для получен...
6,"Определение ""Дети, оставшиеся без попечения ро...","граждане в возрасте до 18 лет, которые осталис...","Каково официальное определение категории ""Дети...","Кто считается детьми, у которых отсутствует по..."
7,"Как узнать назначена ли выплата по услуге ""Пре...",Сведения могут быть отражены в ЭСРН в разделе ...,Где можно проверить информацию о назначении со...,Как узнать о статусе выплаты для детей-сирот п...
8,"Когда возникает право по услуге ""Предоставлени...",с месяца возникновения права на предоставление...,Когда детям-сиротам предоставляется право на с...,В какие сроки следует подать заявление для пол...
9,Какие документы удостоверяют личность по 583 у...,паспорт гражданина РФ;\nвременное удостоверени...,Какие документы подтверждают идентичность при ...,Какие идентификационные документы требуются дл...


In [91]:
# 1.1 TF-IDF Embeddings with a trained vectorizer
def get_tfidf_embeddings_with_vectorizer(data, vectorizer):
    tfidf_matrix = vectorizer.transform(data)
    return tfidf_matrix

# Assuming df_short is the dataset and previous functions are defined
original_questions = df_test['Оригинал вопроса'].values
reframed_questions_1 = df_test['Переформулированный 1'].values
reframed_questions_2 = df_test['Переформулированный 2'].values

# Training TF-IDF vectorizer on original questions
tfidf_vectorizer = TfidfVectorizer().fit(original_questions)

# 1. Get embeddings for original questions
tfidf_embeddings_original = tfidf_vectorizer.transform(original_questions)
w2v_embeddings_original = get_word2vec_embeddings(original_questions, w2v_model)

# 2. Get embeddings for reframed questions using the trained vectorizer
tfidf_embeddings_reframed_1 = get_tfidf_embeddings_with_vectorizer(reframed_questions_1, tfidf_vectorizer)
w2v_embeddings_reframed_1 = get_word2vec_embeddings(reframed_questions_1, w2v_model)

tfidf_embeddings_reframed_2 = get_tfidf_embeddings_with_vectorizer(reframed_questions_2, tfidf_vectorizer)
w2v_embeddings_reframed_2 = get_word2vec_embeddings(reframed_questions_2, w2v_model)

# ... [Rest of the code remains unchanged]


In [86]:
original_questions = df_test['Оригинал вопроса'].values
reframed_questions_1 = df_test['Переформулированный 1'].values
reframed_questions_2 = df_test['Переформулированный 2'].values

# 1. Get embeddings for original questions
tfidf_embeddings_original = get_tfidf_embeddings(original_questions)
w2v_embeddings_original = get_word2vec_embeddings(original_questions, w2v_model)



In [87]:
# 2. Get embeddings for reframed questions
tfidf_embeddings_reframed_1 = get_tfidf_embeddings(reframed_questions_1)
w2v_embeddings_reframed_1 = get_word2vec_embeddings(reframed_questions_1, w2v_model)

tfidf_embeddings_reframed_2 = get_tfidf_embeddings(reframed_questions_2)
w2v_embeddings_reframed_2 = get_word2vec_embeddings(reframed_questions_2, w2v_model)


In [92]:

# 3. Find similar original questions for each reframed question
def find_similar_docs(embeddings_original, embeddings_reframed):
    cosine_similarities = cosine_similarity(embeddings_reframed, embeddings_original)
    similar_docs = cosine_similarities.argsort()[:,::-1]
    return similar_docs

tfidf_similar_docs_1 = find_similar_docs(tfidf_embeddings_original, tfidf_embeddings_reframed_1)
w2v_similar_docs_1 = find_similar_docs(w2v_embeddings_original, w2v_embeddings_reframed_1)

tfidf_similar_docs_2 = find_similar_docs(tfidf_embeddings_original, tfidf_embeddings_reframed_2)
w2v_similar_docs_2 = find_similar_docs(w2v_embeddings_original, w2v_embeddings_reframed_2)



In [99]:

# 4. Evaluating with k=10 for example
k = 3
true_docs = [[i] for i in range(len(original_questions))]

tfidf_ndcg_1, tfidf_precision_1, tfidf_recall_1 = evaluate(tfidf_similar_docs_1, true_docs, k)
w2v_ndcg_1, w2v_precision_1, w2v_recall_1 = evaluate(w2v_similar_docs_1, true_docs, k)

tfidf_ndcg_2, tfidf_precision_2, tfidf_recall_2 = evaluate(tfidf_similar_docs_2, true_docs, k)
w2v_ndcg_2, w2v_precision_2, w2v_recall_2 = evaluate(w2v_similar_docs_2, true_docs, k)

print(f"TF-IDF Reframed 1 - NDCG@{k}:", tfidf_ndcg_1)
print(f"TF-IDF Reframed 1 - Precision@{k}:", tfidf_precision_1)
print(f"TF-IDF Reframed 1 - Recall@{k}:", tfidf_recall_1)

print(f"Word2Vec Reframed 1 - NDCG@{k}:", w2v_ndcg_1)
print(f"Word2Vec Reframed 1 - Precision@{k}:", w2v_precision_1)
print(f"Word2Vec Reframed 1 - Recall@{k}:", w2v_recall_1)

print(f"TF-IDF Reframed 2 - NDCG@{k}:", tfidf_ndcg_2)
print(f"TF-IDF Reframed 2 - Precision@{k}:", tfidf_precision_2)
print(f"TF-IDF Reframed 2 - Recall@{k}:", tfidf_recall_2)

print(f"Word2Vec Reframed 2 - NDCG@{k}:", w2v_ndcg_2)
print(f"Word2Vec Reframed 2 - Precision@{k}:", w2v_precision_2)
print(f"Word2Vec Reframed 2 - Recall@{k}:", w2v_recall_2)

TF-IDF Reframed 1 - NDCG@3: 0.8384882922619096
TF-IDF Reframed 1 - Precision@3: 0.3111111111111111
TF-IDF Reframed 1 - Recall@3: 0.9333333333333333
Word2Vec Reframed 1 - NDCG@3: 0.5464263086904791
Word2Vec Reframed 1 - Precision@3: 0.21111111111111108
Word2Vec Reframed 1 - Recall@3: 0.6333333333333333
TF-IDF Reframed 2 - NDCG@3: 0.821821625595243
TF-IDF Reframed 2 - Precision@3: 0.3
TF-IDF Reframed 2 - Recall@3: 0.9
Word2Vec Reframed 2 - NDCG@3: 0.5761859507142916
Word2Vec Reframed 2 - Precision@3: 0.22222222222222218
Word2Vec Reframed 2 - Recall@3: 0.6666666666666666


In [95]:
k = 3

In [122]:
tfidf_map = mean_avg_precision_at_k(tfidf_similar_docs_1, true_docs,3)
w2v_map = mean_avg_precision_at_k(w2v_similar_docs_1, true_docs,3)

print("TF-IDF MAP Reframed 1:", tfidf_map)
print("Word2Vec MAP Reframed 1:", w2v_map)

TF-IDF MAP Reframed 1: 0.8055555555555555
Word2Vec MAP Reframed 1: 0.5166666666666667


In [121]:
tfidf_map = mean_avg_precision_at_k(tfidf_similar_docs_2, true_docs,3)
w2v_map = mean_avg_precision_at_k(w2v_similar_docs_2, true_docs,3)

print("TF-IDF MAP Reframed 2:", tfidf_map)
print("Word2Vec MAP Reframed 2:", w2v_map)

TF-IDF MAP Reframed 2: 0.7944444444444444
Word2Vec MAP Reframed 2: 0.5444444444444445


## Transformers

In [53]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00


In [55]:
import torch

In [56]:
from transformers import AutoModel, AutoTokenizer

def get_transformers_embeddings(data, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    embeddings = []
    for text in data:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

# Load embeddings
embeddings_multilingual_e5_large = get_transformers_embeddings(original_questions, "intfloat/multilingual-e5-large")



In [57]:
# 2. Get embeddings for reframed questions using the transformer models
embeddings_reframed_1_multilingual_e5_large = get_transformers_embeddings(reframed_questions_1, "intfloat/multilingual-e5-large")


embeddings_reframed_2_multilingual_e5_large = get_transformers_embeddings(reframed_questions_2, "intfloat/multilingual-e5-large")

# 3. Find similar original questions for each reframed question for each model
similar_docs_1_multilingual_e5_large = find_similar_docs(embeddings_multilingual_e5_large, embeddings_reframed_1_multilingual_e5_large)


similar_docs_2_multilingual_e5_large = find_similar_docs(embeddings_multilingual_e5_large, embeddings_reframed_2_multilingual_e5_large)


# 4. Evaluating with k=10 for example
k = 3
true_docs = [[i] for i in range(len(original_questions))]

# Evaluate the models for reframed question 1
ndcg_1_multilingual_e5_large, precision_1_multilingual_e5_large, recall_1_multilingual_e5_large = evaluate(similar_docs_1_multilingual_e5_large, true_docs, k)


# Evaluate the models for reframed question 2
ndcg_2_multilingual_e5_large, precision_2_multilingual_e5_large, recall_2_multilingual_e5_large = evaluate(similar_docs_2_multilingual_e5_large, true_docs, k)

# Print the results for reframed question 1
print("For Reframed Question 1:")
print("Multilingual E5 Large - NDCG@3:", ndcg_1_multilingual_e5_large)


# Print the results for reframed question 2
print("\nFor Reframed Question 2:")
print("Multilingual E5 Large - NDCG@3:", ndcg_2_multilingual_e5_large)



For Reframed Question 1:
Multilingual E5 Large - NDCG@3: 0.9543643251190486

For Reframed Question 2:
Multilingual E5 Large - NDCG@3: 0.9420619835714306


In [112]:
def dcg(retrieved_docs, relevant_docs, k):
    """Compute the Discounted Cumulative Gain."""
    dcg_score = 0
    for i, doc in enumerate(retrieved_docs[:k]):
        if doc in relevant_docs:
            dcg_score += 1 / np.log2(i + 2)  # i starts from 0, hence we add 2
    return dcg_score


In [113]:
def ndcg_at_k(retrieved_docs, relevant_docs, k):
    dcg_val = dcg(retrieved_docs, relevant_docs, k)
    idcg_val = dcg(sorted(relevant_docs, reverse=True), relevant_docs, k)
    if idcg_val == 0:
        return 0
    return dcg_val / idcg_val


In [114]:
def precision_at_k(retrieved_docs, relevant_docs, k):
    """Compute the Precision at k."""
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = [doc for doc in retrieved_k if doc in relevant_docs]
    return len(relevant_retrieved) / k


In [115]:
def recall_at_k(retrieved_docs, relevant_docs, k):
    """Compute the Recall at k."""
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = [doc for doc in retrieved_k if doc in relevant_docs]
    return len(relevant_retrieved) / len(relevant_docs)


In [116]:
def average_precision(retrieved_docs, relevant_docs):
    """Compute the average precision."""
    ap = 0.0
    rel_count = 0
    for i, doc in enumerate(retrieved_docs):
        if doc in relevant_docs:
            rel_count += 1
            ap += rel_count / (i + 1)
    return ap / min(len(relevant_docs), len(retrieved_docs))


In [117]:
def evaluate(similar_docs, true_docs, k):
    ndcgs, precisions, recalls, average_precisions = [], [], [], []

    for i in range(len(similar_docs)):
        relevant_docs = set(true_docs[i])
        retrieved_docs = similar_docs[i][:k]

        # Compute NDCG@k
        ndcgs.append(ndcg_at_k(retrieved_docs, relevant_docs, k))

        # Compute Precision@k
        precisions.append(precision_at_k(retrieved_docs, relevant_docs, k))

        # Compute Recall@k
        recalls.append(recall_at_k(retrieved_docs, relevant_docs, k))

        # Compute Average Precision
        avg_p = average_precision(retrieved_docs, relevant_docs)
        average_precisions.append(avg_p)

    return np.mean(ndcgs), np.mean(precisions), np.mean(recalls), np.mean(average_precisions)


In [72]:

k = 3

ndcg_1_multilingual_e5_large, precision_1_multilingual_e5_large, recall_1_multilingual_e5_large, map_1_multilingual_e5_large = evaluate(similar_docs_1_multilingual_e5_large, true_docs, k)
ndcg_2_multilingual_e5_large, precision_2_multilingual_e5_large, recall_2_multilingual_e5_large, map_2_multilingual_e5_large = evaluate(similar_docs_2_multilingual_e5_large, true_docs, k)

# Print the results for reframed question 1 using intfloat/multilingual-e5-large model
print("For Reframed Question 1 with intfloat/multilingual-e5-large:")
print("NDCG@3:", ndcg_1_multilingual_e5_large)
#print("Precision@3:", precision_1_multilingual_e5_large)
print("Recall@3:", recall_1_multilingual_e5_large)
print("MAP:", map_1_multilingual_e5_large)

# Print the results for reframed question 2 using intfloat/multilingual-e5-large model
print("\nFor Reframed Question 2 with intfloat/multilingual-e5-large:")
print("NDCG@3:", ndcg_2_multilingual_e5_large)
#print("Precision@3:", precision_2_multilingual_e5_large)
print("Recall@3:", recall_2_multilingual_e5_large)
print("MAP:", map_2_multilingual_e5_large)


For Reframed Question 1 with intfloat/multilingual-e5-large:
NDCG@3: 0.9543643251190486
Recall@3: 0.9666666666666667
MAP: 0.95

For Reframed Question 2 with intfloat/multilingual-e5-large:
NDCG@3: 0.9420619835714306
Recall@3: 0.9666666666666667
MAP: 0.9333333333333333


In [106]:
embeddings_paraphrase_mpnet = get_transformers_embeddings(original_questions, "sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# ... [Rest of the similarity search and evaluation code remains unchanged]

In [109]:
embeddings_reframed_1_paraphrase_mpnet = get_transformers_embeddings(reframed_questions_1, "sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

embeddings_reframed_2_paraphrase_mpnet = get_transformers_embeddings(reframed_questions_2, "sentence-transformers/paraphrase-multilingual-mpnet-base-v2")


In [110]:

# 3. Find similar original questions for each reframed question for each model
similar_docs_1_paraphrase_mpnet = find_similar_docs(embeddings_paraphrase_mpnet, embeddings_reframed_1_paraphrase_mpnet)

similar_docs_2_paraphrase_mpnet = find_similar_docs(embeddings_paraphrase_mpnet, embeddings_reframed_2_paraphrase_mpnet)

In [118]:
# Рассчитываем метрики для первого переформулированного вопроса
ndcg_1_paraphrase_mpnet, precision_1_paraphrase_mpnet, recall_1_paraphrase_mpnet, map_1_paraphrase_mpnet = evaluate(similar_docs_1_paraphrase_mpnet, true_docs, k)
map_1_paraphrase_mpnet = mean_avg_precision_at_k(similar_docs_1_paraphrase_mpnet, true_docs, k)

# Рассчитываем метрики для второго переформулированного вопроса
ndcg_2_paraphrase_mpnet, precision_2_paraphrase_mpnet, recall_2_paraphrase_mpnet, map_2_paraphrase_mpnet = evaluate(similar_docs_2_paraphrase_mpnet, true_docs, k)
map_2_paraphrase_mpnet = mean_avg_precision_at_k(similar_docs_2_paraphrase_mpnet, true_docs, k)

# Выводим результаты для первого переформулированного вопроса
print("For Reframed Question 1 with paraphrase-multilingual-mpnet-base-v2:")
print("NDCG@3:", ndcg_1_paraphrase_mpnet)
print("Precision@3:", precision_1_paraphrase_mpnet)
print("Recall@3:", recall_1_paraphrase_mpnet)
print("MAP@3:", map_1_paraphrase_mpnet)

# Выводим результаты для второго переформулированного вопроса
print("\nFor Reframed Question 2 with paraphrase-multilingual-mpnet-base-v2:")
print("NDCG@3:", ndcg_2_paraphrase_mpnet)
print("Precision@3:", precision_2_paraphrase_mpnet)
print("Recall@3:", recall_2_paraphrase_mpnet)
print("MAP@3:", map_2_paraphrase_mpnet)


For Reframed Question 1 with paraphrase-multilingual-mpnet-base-v2:
NDCG@3: 0.9253953169047638
Precision@3: 0.32222222222222224
Recall@3: 0.9666666666666667
MAP@3: 0.9111111111111111

For Reframed Question 2 with paraphrase-multilingual-mpnet-base-v2:
NDCG@3: 0.8964263086904791
Precision@3: 0.3111111111111111
Recall@3: 0.9333333333333333
MAP@3: 0.8833333333333333


In [119]:
embeddings_LaBSE = get_transformers_embeddings(original_questions, "cointegrated/LaBSE-en-ru")

embeddings_reframed_1_LaBSE = get_transformers_embeddings(reframed_questions_1, "cointegrated/LaBSE-en-ru")


embeddings_reframed_2_LaBSE = get_transformers_embeddings(reframed_questions_2, "cointegrated/LaBSE-en-ru")

# 3. Find similar original questions for each reframed question for each model

similar_docs_1_LaBSE = find_similar_docs(embeddings_LaBSE, embeddings_reframed_1_LaBSE)

similar_docs_2_LaBSE = find_similar_docs(embeddings_LaBSE, embeddings_reframed_2_LaBSE)

In [120]:
# Рассчитываем метрики для первого переформулированного вопроса для LaBSE
ndcg_1_LaBSE, precision_1_LaBSE, recall_1_LaBSE, _ = evaluate(similar_docs_1_LaBSE, true_docs, k)
map_1_LaBSE = mean_avg_precision_at_k(similar_docs_1_LaBSE, true_docs, k)

# Рассчитываем метрики для второго переформулированного вопроса для LaBSE
ndcg_2_LaBSE, precision_2_LaBSE, recall_2_LaBSE, _ = evaluate(similar_docs_2_LaBSE, true_docs, k)
map_2_LaBSE = mean_avg_precision_at_k(similar_docs_2_LaBSE, true_docs, k)

# Выводим результаты для первого переформулированного вопроса для LaBSE
print("For Reframed Question 1 with cointegrated/LaBSE-en-ru:")
print("NDCG@3:", ndcg_1_LaBSE)
print("Precision@3:", precision_1_LaBSE)
print("Recall@3:", recall_1_LaBSE)
print("MAP@3:", map_1_LaBSE)

# Выводим результаты для второго переформулированного вопроса для LaBSE
print("\nFor Reframed Question 2 with cointegrated/LaBSE-en-ru:")
print("NDCG@3:", ndcg_2_LaBSE)
print("Precision@3:", precision_2_LaBSE)
print("Recall@3:", recall_2_LaBSE)
print("MAP@3:", map_2_LaBSE)


For Reframed Question 1 with cointegrated/LaBSE-en-ru:
NDCG@3: 0.9543643251190486
Precision@3: 0.32222222222222224
Recall@3: 0.9666666666666667
MAP@3: 0.95

For Reframed Question 2 with cointegrated/LaBSE-en-ru:
NDCG@3: 0.8964263086904791
Precision@3: 0.3111111111111111
Recall@3: 0.9333333333333333
MAP@3: 0.8833333333333333



\documentclass{article}
\usepackage{tabularx}
\usepackage{colortbl}

\definecolor{bestcolor}{rgb}{0.88,1,0.88}

\begin{document}

\begin{table}
\centering
\begin{tabular}{|l|c|c|c|}
\hline
Model & NDCG@3 & Recall@3 & MAP@3 \\
\hline
\multicolumn{4}{|c|}{Reframed 1} \\
\hline
TF-IDF & 0.8385 & 0.9333 & 0.8056 \\
Word2Vec & 0.5464 & 0.6333 & 0.5167 \\
E5 Large & \cellcolor{bestcolor}0.9544 & \cellcolor{bestcolor}0.9667 & \cellcolor{bestcolor}0.95 \\
Paraphrase MPNet & 0.9254 & 0.9667 & 0.9111 \\
LaBSE & \cellcolor{bestcolor}0.9544 & \cellcolor{bestcolor}0.9667 & \cellcolor{bestcolor}0.95 \\
\hline
\multicolumn{4}{|c|}{Reframed 2} \\
\hline
TF-IDF & 0.8218 & 0.9 & 0.7944 \\
Word2Vec & 0.5762 & 0.6667 & 0.5444 \\
E5 Large & \cellcolor{bestcolor}0.9421 & \cellcolor{bestcolor}0.9667 & \cellcolor{bestcolor}0.9333 \\
Paraphrase MPNet & 0.8964 & 0.9333 & 0.8833 \\
LaBSE & 0.8964 & 0.9333 & 0.8833 \\
\hline
\end{tabular}
\caption{Evaluation metrics for different embeddings on reframed questions.}
\end{table}

\end{document}
